In [ ]:
# This notebook shows how to use the graphframes library in order to perform connected components at scale.

In [ ]:
#run this on console to install graphframes 0.8.2 (works with spark >= 3.1.1 )
#pip install "git+https://github.com/mamonu/graphframes.git@master#egg=graphframes&subdirectory=python"

In [ ]:

import pyspark
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [ ]:
#Using splink_graph.util functions to find where splink_graph is installed and create a
#suitable string to put on spark.jars configuration so graphframes work.

from splink_graph.utils import _get_local_site_packages_dir, _create_spark_jars_string

localsite=_get_local_site_packages_dir()
localsite

sjstring = _create_spark_jars_string().replace(" ", "")
sjstring

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Configure SparkConf object 
conf = SparkConf()
conf.set("spark.jars", sjstring)
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set('spark.driver.extraClassPath', sjstring)


In [ ]:
# Create SparkSession with the SparkConf
spark = SparkSession.builder.appName("GraphFramesExample") \
    .config(conf=conf) \
    .getOrCreate()

In [ ]:
spark.sparkContext._conf.get("spark.jars")

In [ ]:
spark.version

In [ ]:
from graphframes import *


In [ ]:
# Create a Vertex DataFrame with unique ID column "id"
v = spark.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "Dennis", 30),    
  ("e", "Ernest", 30), 
  ("f", "Ford", 30),
  ("g", "George", 30), 
  ("h", "Harry", 30),
  ("i", "Ion", 30),
  ("j", "John", 30),
    
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = spark.createDataFrame([
  ("a", "b", 0.4),
  ("b", "c", 0.56),
 
  
  ("d", "e", 0.84),
  ("e", "f", 0.65),
  ("f", "d", 0.67),
  ("f", "g", 0.34),
  ("g", "h", 0.99),
  ("h", "i", 0.5),
  ("h", "j", 0.8),]
    
, ["src", "dst", "similarity"])
# Create a GraphFrame

g = GraphFrame(v, e)

In [ ]:
# Check if graphframes library works
g.inDegrees.show()